# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****tM0A
LANGSMITH_API_KEY=****bc46
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


## Human👨‍💻 and AI 🤖 Messages

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-5-nano", 
    system_prompt="You are a full-stack comedian"
)

In [3]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [4]:
print(result["messages"][-1].content)

Hey there! I'm doing great—running at 100% CPU, fueled by caffeine, and debugging punchlines all day. How about you? Want a quick frontend joke, a backend joke, or a SQL pun to spice things up?


In [5]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I'm doing great—running at 100% CPU, fueled by caffeine, and debugging punchlines all day. How about you? Want a quick frontend joke, a backend joke, or a SQL pun to spice things up?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [7]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [8]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Baseball, a diamond stitched in grass and sun.
Nine innings, a heartbeat between the lines.
A pitcher and catcher; a quiet duel behind the plate.
Three strikes, four balls—the count that tests the breath.
The fastball rides the corner; the curve dives to dirt.
A bat crackles; the ball climbs the summer sky.
Gloves whisper, outs tally, chalk lines witness.
Nine innings grow into a season; seasons breed legends.
The World Series crowns the dream with a roar.
Baseball: old as summers, sharp as a chalk line.

Want a quick primer on rules, positions, or history?


#### Dictionaries

In [9]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Spikes bite morning air
Lanes tremble with each deep breath
Finish lines erupt


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [10]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [11]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [12]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Dawn field, cleats whisper
Breath huddles in cold bright air
Ball arcs, crowd as wind


In [13]:
result["messages"][-1].content

'Dawn field, cleats whisper\nBreath huddles in cold bright air\nBall arcs, crowd as wind'

In [14]:
print(len(result["messages"]))

4


In [15]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_pFqyJfjPlrXUSpKf3AyLBD2K)
 Call ID: call_pFqyJfjPlrXUSpKf3AyLBD2K
  Args:
    text: Dawn field, cleats whisper
Breath huddles in cold bright air
Ball arcs, crowd as wind
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Dawn field, cleats whisper
Breath huddles in cold bright air
Ball arcs, crowd as wind


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [16]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='f29872b9-eec9-4fc6-bb30-f94fff17c3a7'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 817, 'prompt_tokens': 170, 'total_tokens': 987, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ca8tO3Io6u7sixdW8LN57INltANd2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--61ea4ebc-83c3-4a70-8cb5-4aa8a4816388-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Dawn field, cleats whisper\nBreath huddles in cold bright air\nBall arcs, crowd as wind'}, 'id': 'call_pFqyJfjPlrXUSpKf3AyLBD2K', 'type': 'tool_call'}

You can select just the last message, and you can see where the final message is coming from.

In [17]:
result["messages"][-1]

AIMessage(content='Dawn field, cleats whisper\nBreath huddles in cold bright air\nBall arcs, crowd as wind', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 237, 'total_tokens': 264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ca8tafrhE6qmXVQHGFhF6JJ03bQeW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f275cbbe-eca1-4ec8-8120-3329c35792e2-0', usage_metadata={'input_tokens': 237, 'output_tokens': 27, 'total_tokens': 264, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
result["messages"][-1].usage_metadata

{'input_tokens': 237,
 'output_tokens': 27,
 'total_tokens': 264,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [19]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 27,
  'prompt_tokens': 237,
  'total_tokens': 264,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-Ca8tafrhE6qmXVQHGFhF6JJ03bQeW',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [24]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are an expert in writing haikus about Gabriel Garcia Márquez.",
)

msg = HumanMessage("Write me a haiku about Gabriel Garcia Márquez.")
result = agent.invoke({"messages": msg})

checking haiku, it has 3 lines:
 Macondo hums low
García Márquez dreams rain
yellow flowers fall


In [25]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Write me a haiku about Gabriel Garcia Márquez.
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_l4mSlXsJFxt5GjWJPlvcmphR)
 Call ID: call_l4mSlXsJFxt5GjWJPlvcmphR
  Args:
    text: Macondo hums low
García Márquez dreams rain
yellow flowers fall
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Macondo hums low
García Márquez dreams rain
yellow flowers fall
